# steps in the process : 
* Compile a C file into a **shared object library**
* write a **python wrapper** for the shared object library
    * attaching **type signatures** to c function attributes and returns
    * defining **python wrapper function** for the corresponding C functions\
    * defining **classes** to represent **C data types** in python
    * defining **classes** to represent **Structures in C**

# setup
locating the working directory

In [1]:
%%file locate_file.py

def locate_file() : 
    working_directory = ''
    for element in __file__.split('/')[:-1] :
        working_directory += element + '/'
    return working_directory

Overwriting locate_file.py


In [2]:
import locate_file
working_directory = locate_file.locate_file()

import os
os.chdir(working_directory)

# C file

In [3]:
%%file sample.c
#include <math.h>
#include "Python/Python.h"

int divide(int a,int b,int *remainder) {
    int quot = a/b;
    *remainder = a%b;
    return quot;
}

void avg(double *a,int n) {
    int i;
    double total = 0.0;
    for(i=0;i<n;i++) {
        //total += *(a+i);
        *(a+i) += 1;
    }
    //return total/n;
}

Overwriting sample.c


# Compiling the C file into a shared object library

In [4]:
os.system('cd '+ working_directory)
!clang -c sample.c
!clang -o libsample.so -shared sample.o
!file libsample.so

libsample.so: Mach-O 64-bit dynamically linked shared library x86_64


# Python extension for wrapping the shared object library

### *loading the shared object library*

In [5]:
import ctypes
_sample_ = ctypes.cdll.LoadLibrary(working_directory + 'libsample.so')

### *attaching type signatures to C function attributes and return values*

the *.argtypes* attribute is a tuple containint the input arguments to a fuction

In [6]:
_sample_.divide.argtypes = [ctypes.c_int,ctypes.c_int,ctypes.POINTER(ctypes.c_int)]

*.restype* is the return type of the function

In [7]:
_sample_.divide.restype = ctypes.c_int

Attaching the type signatures is critical if you want to make Python pass the right kinds of arguments and convert data correctly. If you don't do this, not only will the code work, but you might cause the entire interpreter process to crash.

### python wrapper function for the corresponding C function

In [8]:
def divide(x,y) : 
    #for arguments involving pointers, you ususlly have to construct a
    #compatible ctypes object and pass it in like this
    remainder = ctypes.c_int()
    quotient = _sample_.divide(x,y,remainder)
    return quotient,remainder.value

In [9]:
divide(2,3)

(0, 2)

### python classes for defining types in C

<p>for the avg() function, the underlying C code expects to receive a pointer and a length representing an array. However, from the python side, we must consider the following questions :</p> 
* what is an array?
* Is it a list, or a tuple, or a numpy array?
* Is it all of these?

<p>In practice, you would want an array to take multiple forms, and support multiple possibilities.</p>
In such situations, we write a python class that makes this possible.

<p>In our case, we write the **DoubleArrayType class** to handle this situation.</p>
The *from_param()* method is defined to take a single parameter and then narrow it down to compatible ctypes object. (the name "*from_param*" is important, since the ctypes module searches for the method **with this name** inside the defined class, when such a class is set as *.argtypes* or *.restypes* attribute for a function, as a part of typechecking)

In [10]:
class DoubleArrayType() : 
    
    def from_param(self,param) : 
        typename = type(param).__name__
        if hasattr(self,'from_'+typename) : 
            return getattr(self,'from_'+typename)(param)
        elif isinstance(param,ctypes.Array) : 
            return param
        else : 
            raise TypeError('Can not convert %s' %typename)

The corresponding functions then convert the valid input datatype, C compatible ctypes object

In [20]:
import numpy

def from_list(self,param) : 
#     val = ((ctypes.c_double)*len(param))(*param) #converts into a c_int_Array object
#     return val
    c_array = numpy.ascontiguousarray(param,dtype=int,order='C')
    pointer = c_array.ctypes
    return pointer
DoubleArrayType.from_list = from_list

def from_tuple(self,param) : 
    return self.from_list(param)
DoubleArrayType.from_tuple = from_tuple

def from_ndarray(self,param) : 
    return param.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
#     return self.from_list(param)
DoubleArrayType.from_ndarray = from_ndarray

Now, setting the DoubleArray class object as one of the **argtypes** for *_sample_.avg* function

In [21]:
DoubleArray = DoubleArrayType()
_sample_.avg.argtypes = [DoubleArray,ctypes.c_int]
# _sample_.avg.restype = ctypes.c_double
_sample_.avg.restype = ctypes.c_void_p

The last thing to do is to define a wrapper function for the *_sample_.avg* C function

In [22]:
def avg(values) : 
    return _sample_.avg(values,len(values))

In [23]:
import numpy as np
arr = [1.0,2.0,3.0,4.0,5.0]
avg(arr)
arr

ArgumentError: argument 1: <class 'TypeError'>: ascontiguousarray() got an unexpected keyword argument 'order'